In [99]:
import tensorflow as tf
import pandas as pd

df_credb = pd.read_excel('createDebate.xlsx')
df_credb = df_credb.query("stance == 'prefers strict gun control' | stance == 'opposes strict gun control'")
df_credb = df_credb[["text", "stance"]]
#Encoding the stance, which is our target class
import numpy as np

if type(df_credb.stance.iloc[1]) is str:
    class_mapping = {label: idx for idx, label in 
                    enumerate(np.unique(df_credb['stance']))}
#Prevents accidentally overwriting the 
if type(df_credb.stance.iloc[1]) is str:
    df_credb['stance'] = df_credb['stance'].map(class_mapping)
    
oppose = df_credb.query("stance == 0")
support = df_credb.query("stance == 1")

oppose = oppose[:100]
support = support[:100]

df_reduced = pd.concat([support,oppose])

In [100]:
#import the BERT model and tokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from transformers import TFBertModel, BertTokenizer

if 'bertModel' not in locals() or 'tokenizer' not in locals():
    #BERT base model
    bertModel = TFBertModel.from_pretrained('bert-base-uncased')
    
    #BERT Tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [101]:
# Define some example data
texts = df_credb.text.values.tolist()
labels = df_credb.stance.values.tolist()

#tokenize the texts
tokens = tokenizer(texts, padding=True, truncation=True, return_tensors='np') #Use Tensorflow for faster training

#Create dict of tokenized data
tokenized_data = dict(tokens)

#Encode labels as a numpy array
labels = np.array(labels)

In [107]:
# Define the input layer for the model
input_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name='attention_mask')

bert_layer = bertModel({'input_ids': input_ids, 'attention_mask': attention_mask})[0]

# Add a GlobalMaxPooling1D layer to reduce the sequence length dimension to 1
pooled_output = tf.keras.layers.GlobalMaxPooling1D()(bert_layer)

dropout = tf.keras.layers.Dropout(0.1, name="dropout")(pooled_output)

output_layer = tf.keras.layers.Dense(1,activation='sigmoid')(dropout)

'''dense_layer_2 = tf.keras.layers.Dense(256,activation='relu')(dense_layer_1)

output_layer = tf.keras.layers.Dense(2,activation='sigmoid')(dense_layer_2)'''

for layer in model.layers[:3]:
    layer.trainable = False

model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output_layer)

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer=Adam(3e-5),
                 loss = 'mean_squared_error',
                 metrics = METRICS)


#model.fit(tokenized_data, labels)

In [108]:
model.summary()

Model: "model_29"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    multiple             109482240   ['attention_mask[0][0]',         
                                                                  'input_ids[0][0]']              
                                                                                                  
 global_max_pooling1d_17 (Globa  (None, 768)         0           ['tf_bert_model[31][0]']  

In [109]:
model.fit(tokenized_data, labels, batch_size=16, epochs=2,verbose=1)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


/Users/einsteinium/opt/anaconda3/envs/tf/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['token_type_ids'] which did not match any model input. They will be ignored by the model.
  if mask is None:


382/382 [==============================] - 2196s 6s/step - loss: 0.2806 - accuracy: 0.4923 - precision: 0.4936 - recall: 0.5913
Epoch 2/2
382/382 [==============================] - 2259s 6s/step - loss: 0.2695 - accuracy: 0.5046 - precision: 0.5047 - recall: 0.4926
